# Import dependencies.

In [ ]:
import warnings

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.datasets import mnist
from keras.utils import to_categorical, disable_interactive_logging
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

from datetime import datetime
from skimage.transform import resize

# Suppress specific warning message
warnings.filterwarnings(
    action='ignore',
    message='The name tf.executing_eagerly_outside_functions is deprecated.*',
)
# warnings.filterwarnings("ignore")
# disable_interactive_logging();

In [ ]:
# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Resize images to match the required input size of DenseNet (32x32)
train_images_resized = np.array([resize(img, (32, 32)) for img in train_images])
test_images_resized = np.array([resize(img, (32, 32)) for img in test_images])

# Normalize pixel values to be between 0 and 1
train_images_resized = train_images_resized.astype('float32') / 255
test_images_resized = test_images_resized.astype('float32') / 255

# Replicate single channel across three channels to match DenseNet input shape
train_images_resized = np.repeat(train_images_resized[..., np.newaxis], 3, axis=-1)
test_images_resized = np.repeat(test_images_resized[..., np.newaxis], 3, axis=-1)

# One-hot encode labels
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
# Load pre-trained DenseNet model without the top layer
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Combine the base model with custom layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Create a TensorBoard callback
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tboard_callback = TensorBoard(
    log_dir = logs,
    histogram_freq = 1,
    profile_batch = '500,520'
)
# Train the model
history = model.fit(
    train_images_resized,
    train_labels,
    epochs=3,
    batch_size=128,
    validation_data=(test_images_resized, test_labels),
    callbacks = [tboard_callback]
)


In [ ]:
# Load the TensorBoard notebook extension.
# %reload_ext tensorboard
# %load_ext tensorboard

# Launch TensorBoard and navigate to the Profile tab to view performance profile
# %tensorboard --logdir=logs
%tensorboard --logdir=logs --port=6006

In [ ]:
# Plot training & validation loss values using Seaborn

sns.lineplot(x=range(1, len(history.history['loss']) + 1), y=history.history['loss'], label='Train')
sns.lineplot(x=range(1, len(history.history['val_loss']) + 1), y=history.history['val_loss'], label='Test')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
# plt.show();

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images_resized, test_labels)
print('Test accuracy:', test_acc)

# Getting data, observations.
## Get dataset, split data into train and test sets.